In [1]:
import time
start_time = time.time()

In [2]:
import pandas as pd
import os
from bidi.algorithm import get_display
import re
import multiprocessing

In [3]:
def send_email(user='dsakaidf@gmail.com', pwd='d54k4idf!', recipient='shkasta@post.bgu.ac.il',
               subject='finish expirement', body='finish the expirement'):
    import smtplib

    gmail_user = user
    gmail_pwd = pwd
    FROM = user
    TO = recipient if type(recipient) is list else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        # SMTP_SSL Example
        server_ssl = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server_ssl.ehlo()  # optional, called by login()
        server_ssl.login(gmail_user, gmail_pwd)
        # ssl server doesn't support or need tls, so don't call server_ssl.starttls()
        server_ssl.sendmail(FROM, TO, message)
        # server_ssl.quit()
        server_ssl.close()
        print('successfully sent the mail')
    except:
        print("failed to send mail")


In [4]:
import pickle

In [5]:
file_type = 'thread'
field_name = 'title'

In [6]:
pickle_file = open(f'{file_type}_{field_name}_token_lists.pickle', "rb" )
corpus = pickle.load( pickle_file )
pickle_file.close()

In [7]:
##Creating a list of stop words and adding custom stopwords
import codecs
with codecs.open('stopwords.txt', 'r', encoding='utf8') as f:
    text = f.read()
stop_words = set(word for word in text.split('\r\n'))

In [8]:
corpus_temp = list()
for sentence in corpus:
    sentence_temp = list()
    for word in sentence:
        try:
            if len(word) > 1 and not word in stop_words:
                sentence_temp.append(word.replace('.','').replace(',','').replace('?','').replace('!',''))
        except:
            print(word)
    corpus_temp.append(sentence_temp)
corpus = corpus_temp

In [9]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [10]:
data_lemmatized = corpus

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

id2word.filter_extremes(no_below = 0.05, no_above=0.5)
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [12]:
%time lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=20,random_state=100,update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

CPU times: user 1min 19s, sys: 2 s, total: 1min 21s
Wall time: 1min 22s


In [13]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.224*"יחידה" + 0.203*"קבן" + 0.052*"תור" + 0.019*"יחיד" + 0.000*"קרבי" + '
  '0.000*"מעבר" + 0.000*"גימלים" + 0.000*"מקצוע" + 0.000*"המלצה" + '
  '0.000*"זאת"'),
 (1,
  '0.182*"טופס" + 0.146*"לוחמה" + 0.132*"55" + 0.076*"לקרבי" + 0.026*"סעיף" + '
  '0.012*"מדינה" + 0.000*"אלקטרונית" + 0.000*"מעבר" + 0.000*"תקשוב" + '
  '0.000*"גוב"'),
 (2,
  '0.254*"קבע" + 0.050*"שנתיים" + 0.036*"ראיונות" + 0.033*"מגן" + '
  '0.000*"מילואים" + 0.000*"תנאי" + 0.000*"רגילים" + 0.000*"בדלים" + '
  '0.000*"חברי" + 0.000*"וגימלאים"'),
 (3,
  '0.169*"גיבוש" + 0.147*"טירונות" + 0.131*"צנחנים" + 0.058*"צבאית" + '
  '0.039*"מלשב" + 0.030*"02" + 0.028*"מכינה" + 0.020*"קדם" + 0.010*"טלפון" + '
  '0.007*"צוללות"'),
 (4,
  '0.469*"גיוס" + 0.055*"מפקד" + 0.055*"חייב" + 0.040*"רפואית" + 0.037*"חובש" '
  '+ 0.033*"לוחמים" + 0.021*"ייעוץ" + 0.019*"טחוני" + 0.010*"תואר" + '
  '0.000*"תאריך"'),
 (5,
  '0.250*"חיל" + 0.135*"שיבוץ" + 0.119*"אוויר" + 0.106*"לוחם" + 0.071*"שחרור" '
  '+ 0.044*"להתגייס" + 0.043*"פסיכ

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -25.07558411790241

Coherence Score:  0.5052145430220663


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.354869  0.114937       1        1  11.786139
4     -0.098219 -0.339484       2        1   9.594008
3      0.012832  0.009164       3        1   8.639336
9     -0.001543  0.001652       4        1   8.323586
19    -0.016827 -0.008960       5        1   7.277691
17     0.017008  0.010930       6        1   6.670805
12     0.008415  0.007018       7        1   6.663404
14     0.026599  0.014471       8        1   5.455022
13     0.031399  0.015873       9        1   5.194026
16     0.020565  0.012308      10        1   4.376591
7      0.023962  0.013551      11        1   4.370746
1      0.028168  0.014951      12        1   3.535777
0      0.035088  0.016663      13        1   3.431657
18     0.037691  0.016928      14        1   3.088496
11     0.036606  0.016907      15        1   3.050043
8      0.038764  0.016599      16        1   2.205826
10     0.038393  0.016838      17        1   1.761693
15     0.038559  0.016736      18        1   1.678043
2      0.038572  0.016756      19        1   1.553163
6      0.038835  0.016161      20        1   1.343954, topic_info=           Term         Freq        Total Category  logprob  loglift
65         גיוס  3788.000000  3788.000000  Default  30.0000  30.0000
9         תפקיד  2331.000000  2331.000000  Default  29.0000  29.0000
92       פרופיל  2240.000000  2240.000000  Default  28.0000  28.0000
29          חיל  2480.000000  2480.000000  Default  27.0000  27.0000
200        קורס  1774.000000  1774.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
794        שלדג     0.061627     2.428161  Topic20  -9.8166   0.6358
12906   קרביות/     0.061625     2.428076  Topic20  -9.8166   0.6358
12902    דורשות     0.061625     2.428076  Topic20  -9.8166   0.6358
12905  מבצעיות:     0.061625     2.428076  Topic20  -9.8166   0.6358
912      דובדבן     0.061627     2.428279  Topic20  -9.8166   0.6357

[703 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
155       3  0.990422       02
112      15  0.977533       10
149      12  0.994265       55
91        4  0.993356       64
49        9  0.888729   אדומות
...     ...       ...      ...
205       7  0.998177  תותחנים
195       8  0.986372    תכנות
124       9  0.981147    תעודת
9         6  0.998873    תפקיד
250       4  0.997052  תפקידים

[120 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 4, 10, 20, 18, 13, 15, 14, 17, 8, 2, 1, 19, 12, 9, 11, 16, 3, 7])

In [16]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3,print_each_graph=False):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        #print(f'calc with {num_topics} done')
        ##### Show graph
        if print_each_graph:
            x = range(start, num_topics+1, step)
            plt.plot(x, coherence_values)
            plt.xlabel("Num Topics")
            plt.ylabel("Coherence score")
            plt.legend(("coherence_values"), loc='best')
            plt.show()
    return model_list, coherence_values

In [17]:
start = 50
limit = 70
step = 1

In [18]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)

KeyboardInterrupt: 

In [ ]:
# Show graph
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

send_email()

In [ ]:
from gensim.models import HdpModel
hdp_model = HdpModel(corpus, id2word)

In [ ]:
# Compute Perplexity
#print('\nPerplexity: ', hdp_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=hdp_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Print the Keyword in the 10 topics
pprint(hdp_model.print_topics())
doc_lda = hdp_model[corpus]

In [ ]:
elapsed_time = time.time() - start_time
print(elapsed_time)